In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import requests
import re
from bs4 import BeautifulSoup
import json
import time
import random
import datetime
from threading import Timer

In [9]:
def search(goods_id):
    url = "https://buff.163.com/api/market/goods/sell_order?game=csgo&goods_id="+goods_id+"&page_num=1&sort_by=default&mode=&allow_tradable_cooldown=1"
    headers={}
    html = requests.get(url,headers = headers).text
    return html

volumes = {}
num = len(ak_goods)+len(a4_goods)+len(a1_goods)+len(awp_goods)+len(boxs)
for i in range(num):
    if(i<len(ak_goods)):
        volumes[ak_goods[i]] = []
    elif(i<len(ak_goods)+len(a4_goods)):
        volumes[a4_goods[i-len(ak_goods)]] = []
    elif(i<len(ak_goods)+len(a4_goods)+len(awp_goods)):
        volumes[awp_goods[i-len(ak_goods)-len(a4_goods)]] = []
    elif(i<len(ak_goods)+len(a4_goods)+len(awp_goods)+len(boxs)):
        volumes[boxs[i-len(ak_goods)-len(a4_goods)-len(awp_goods)]] = []
def search_volume(market_name,itemid):
    #判断有无历史的查询
    if(len(volumes['itemid']) == 0):
        has_volume = True
    else:
        has_volume = False
    url = "https://steamcommunity.com/market/priceoverview/?country=JP&currency=8&appid=730&market_hash_name="+market_name
    html = requests.get(url).text
    if html == "null":
        if(has_volume):
            return volumes['itemid'][2]
        else:
            return '-1'
    volume = [itemid,market_name,json.loads(html)['volume'],str(datetime.date.today())+"-"+time.strftime('%H')]
    volumes['itemid'] = volume
    return volume[2]

In [13]:
ak_goods = [781649,781656,781666,33912,33911,33913,759234,759341,871340,871099,871478,33935,33937,33936,33859,33861,33860,33969,33971,33970,33972,763469,763311,763250,33868,33870,33869,33871,33975,33882,33945,33947,33946,33907,33906,33961,33960,779257,779194,779196]
a4_goods = [35345,35347,769584,769583,769572,35328,35327,45261,45385,45262,35261,35263,35262,857690,857611,857610,35253,35254,35252,35353,35355,35354,35281,35283,35282,781560,781559,781564,835822,835770,835603]
a1_goods = [35185,35187,35186,35190,35192,35191,835780,835800,835624,835932,835645,779233,779198,35238,35240,35239,35209,759220,35197,35199,35198,35212,35214,35213,35235,35234]
awp_goods = [34066,34067,34065,775601,775550,773715,773720,773698,34102,763243,763400,763332,34095,34097,34096,34070,34078,34080,34088,34090,34089,34126,34125,34073,45247,45248,]
boxs = [871092,38150,34987,35895,38148,34989,774681,35086,34369,835343,35883]
#len(ak_goods)+len(a4_goods)+len(a1_goods)+len(awp_goods)+len(boxs) = 134
#for test
# ak_goods = [781649,781656]
# a4_goods = [35345,35347]
# a1_goods = [35185,35187]
# awp_goods = [34066,34067]
def clew(goods,ans):
    for i in range(len(goods)):
        goods_id = str(goods[i])
        soup = search(goods_id)
        string = str(soup.p)
        for j in range(len(string)):
            if string[j] == "{":
                break
        start = j
        for j in reversed(range(len(string))):
            if string[j] == "}":
                break
        end = j+1
        dejson = json.loads(str(soup.p)[start:end])
        dejson['data']['goods_infos'][goods_id]
        name = dejson['data']['goods_infos'][goods_id]['name']
        price_buff_mean = np.mean([float(i['price']) for i in dejson['data']['items']])
        price_steam = float(dejson['data']['goods_infos'][goods_id]['steam_price_cny'])
        money = (price_steam/1.15)-price_buff_mean
        ans.append([name,price_buff_mean,price_steam,money,money/price_buff_mean])
        time.sleep(random.randint(2,5))
    return ans

In [9]:
def takelast(elem):
    return elem[-1]

In [104]:
ans = []
ans = clew(ak_goods,ans)
ans = clew(a4_goods,ans)
ans = clew(a1_goods,ans)
ans = clew(awp_goods,ans)
ans.sort(key=takelast, reverse=True)
ans = pd.DataFrame(columns=["饰品名","buff均价","steam市场价","利润","利率"],data=ans)
ans.to_csv("buff/"+str(datetime.date.today())+"-"+time.strftime('%H')+".csv",encoding='utf-8')

In [13]:
print(ans)
ans = []

[['AK-47 | 阿努比斯军团 (崭新出厂)', 123.696, 178.7, 31.695304347826095, 0.25623548334486235], ['AK-47 | 阿努比斯军团 (略有磨损)', 78.774, 115.32, 21.504260869565215, 0.27298678332400556], ['AK-47 | 阿努比斯军团 (久经沙场)', 59.39000000000001, 88.77, 17.801304347826083, 0.2997357189396545], ['AK-47 | 燃料喷射器 (略有磨损)', 536.4300000000001, 795.73, 155.5091304347826, 0.2898964085431139]]


In [63]:
ans

[['AK-47 | 阿努比斯军团 (崭新出厂)',
  117.68800000000002,
  159.8,
  21.26852173913045,
  0.18071954438116414],
 ['AK-47 | 阿努比斯军团 (略有磨损)',
  77.57900000000001,
  111.57,
  19.438391304347817,
  0.25056254017643714],
 ['AK-47 | 阿努比斯军团 (久经沙场)',
  58.83399999999999,
  89.32,
  18.83556521739132,
  0.32014762241886197]]

In [14]:
url = "https://buff.163.com/api/market/goods/sell_order?game=csgo&goods_id=798055&page_num=1&sort_by=default&mode=&allow_tradable_cooldown=1"
headers={"Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9","Accept-Encoding":"gzip, deflate, br","Accept-Language":"zh-CN,zh;q=0.9","Cache-Control":"max-age=0","Connection":"keep-alive","Cookie":"_ns=NS1.2.1049642886.1623160927; _ntes_nuid=5df6afdb0996c600ed011100d0281097; _ntes_nnid=1389cf205da35139605e9b364882ff49,1623500252458; NTES_CMT_USER_INFO=311781205%7C%E6%9C%89%E6%80%81%E5%BA%A6%E7%BD%91%E5%8F%8B0iBmtl%7Chttp%3A%2F%2Fcms-bucket.nosdn.127.net%2F2018%2F08%2F13%2F078ea9f65d954410b62a52ac773875a1.jpeg%7Cfalse%7CeWQuMDA0MDcwNDQ3OGQwNGM4M2FAMTYzLmNvbQ%3D%3D; UM_distinctid=17a95ce1f59c5-06d7adb5982e07-6373264-1fa400-17a95ce1f5a4b2; _ga_C6TGHFPQ1H=GS1.1.1626794968.1.0.1626794968.0; Device-Id=TY9XKDqfOjQqsZXOfoeh; Locale-Supported=zh-Hans; _ga=GA1.2.915409640.1623760724; NTES_YD_SESS=P7PGcL2rZhC5Ygo8hdbjGun3fGMFhNxqXz5_l_W0Qbc6RJKSWmhaEEkqMiLQQjm3FUJLPXJrJt71K1ece117GdfplaKzAXdBtEqFKvdL4JrAuqadiNKP.iN6VJfCVIx8siOQOtSBLq4sY8w3W.SciVZVDlAFNEyLqKIit4YGEnIbV4.FCCn8CcCjwaQNIt2fVGqaXd5tXPWqNHqKJN0B4HlBp0ZdX.CiKN_QUfmP9.pw5; S_INFO=1632816004|0|0&60##|81-8078317669; P_INFO=81-8078317669|1632816004|1|netease_buff|00&99|null&null&null#JP&null#10#0|&0||81-8078317669; game=csgo; _gid=GA1.2.1572040392.1633964171; session=1-rOsVu91zf2exUtKpgQi4LfykIKplMUdSY00cj3JMdZk52040520166; csrf_token=IjBkYWQ3YTliNDAyZjI3ZGZjMzhkZjkxOTViYzA1MzkwOWU0MTJlMGMi.FEg44g.nc3SC7fvLVaYDNy_ROlQfIddew4","Host":"buff.163.com","sec-ch-ua":"\"Chromium\";v=\"94\", \"Google Chrome\";v=\"94\", \";Not A Brand\";v=\"99\"","sec-ch-ua-mobile":"?0","sec-ch-ua-platform":"\"Windows\"","Sec-Fetch-Dest":"document","Sec-Fetch-Mode":"navigate","Sec-Fetch-Site":"none","Sec-Fetch-User":"?1","Upgrade-Insecure-Requests":"1","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
html = requests.get(url,headers = headers).text
soup=BeautifulSoup(html)

In [11]:
ans = []
ans = clew([781649,781656,781666,33912],ans)

In [15]:
soup

<html><body><p>{
  "code": "OK", 
  "data": {
    "goods_infos": {}, 
    "has_market_stores": {}, 
    "items": [], 
    "page_num": 1, 
    "page_size": 10, 
    "sort_by": "price.asc", 
    "total_count": 0, 
    "total_page": 1, 
    "user_infos": {}
  }, 
  "msg": null
}
</p></body></html>